In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np
import rpy2.robjects as robjects

/Users/danielle/miniconda2/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: * The project library is out of sync with the lockfile.
* Use `renv::restore()` to install packages recorded in the lockfile.

  warnings.warn(x, RRuntimeWarning)
/Users/danielle/miniconda2/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Warning:
  warnings.warn(x, RRuntimeWarning)
/Users/danielle/miniconda2/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  namespace ‘ggplot2’ is not available and has been replaced
by .GlobalEnv when processing object ‘plot0’

  warnings.warn(x, RRuntimeWarning)
/Users/danielle/miniconda2/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  namespace ‘scales’ is not available and has been replaced
by .GlobalEnv when processing object ‘p4’

  warnings.warn(x, RRuntimeWarning)


In [9]:
# read in mgx data
mgx_genus = pd.read_csv("diabimmune_mgx_genus.csv")
mgx_genus.rename(columns={'taxname': 'taxa'}, inplace=True)

In [8]:
# mgx_genus = mgx[mgx['taxa'].str.contains("\|g__")] # keep genera
# mgx_genus = mgx_genus[~mgx_genus['taxa'].str.contains("\|s__")] # keep species
# mgx_genus["taxa"] = mgx_genus['taxa'].str.split("\|g__").str[-1]
# mgx_genus["taxa"] = mgx_genus['taxa'].str.split("\|s__").str[0]#

In [10]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx_genus = mgx_genus[~mgx_genus.taxa.str.contains("_unclassified")]
mgx_genus = mgx_genus[~mgx_genus.taxa.str.contains("_noname")]
mgx_genus = mgx_genus[~mgx_genus.taxa.str.contains("virus")]
mgx_genus = mgx_genus[~mgx_genus.taxa.str.contains("Candidatus")]
mgx_genus = mgx_genus[~mgx_genus.taxa.str.contains("candidate")]

In [11]:
mgx_genus = mgx_genus.groupby(['taxa']).sum().reset_index() 

In [12]:
mgx_genus.head()

,taxa,G69146,G69147,G69148,G69149,G69150,G69151,G69152,G69153,G69154,...,G80617,G80619,G80620,G80621,G80623,G80624,G86980,G86986,G90143,G90147
0,Absiella,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.013550,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.001091
4,Actinomyces,0.0,0.0,0.0,0.0,0.0,0.0,0.000045,0.0,0.000083,...,0.0,0.0,0.0,0.0,0.0,0.000766,0.0,0.0,0.000011,0.000051


### Looking at families in metagenomic data

In [7]:
mgx_family = mgx[mgx['taxa'].str.contains("\|f__")] # keep families genera
mgx_family = mgx_family[~mgx_family['taxa'].str.contains("\|g__")] # remove genera

mgx_family["taxa"] = mgx_family['taxa'].str.split("\|f__").str[-1]

In [8]:
mgx_family = mgx_family[~mgx_family.taxa.str.contains("_unclassified")]
mgx_family = mgx_family[~mgx_family.taxa.str.contains("_noname")]
mgx_family = mgx_family[~mgx_family.taxa.str.contains("virus")]
mgx_family = mgx_family[~mgx_family.taxa.str.contains("Candidatus")]
mgx_family = mgx_family[~mgx_family.taxa.str.contains("candidate")]

In [9]:
mgx_family = mgx_family.groupby(['taxa']).sum().reset_index() 

In [10]:
mgx_family.head()

,taxa,G69146,G69147,G69148,G69149,G69150,G69152,G69153,G69154,G69155,...,G80612,G80613,G80614,G80615,G80616,G80619,G80620,G80621,G80623,G80624
0,Acetobacteraceae,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,...,0.0,0.00000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.00000
1,Acholeplasmataceae,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,...,0.0,0.00000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.00000
2,Acidaminococcaceae,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.99771,0.0,...,0.0,0.00000,0.00000,4.3552,0.0,0.0,0.0,0.0,0.0,0.00000
3,Acidobacteriaceae,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,...,0.0,0.00000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.00000
4,Actinomycetaceae,0.0,0.0,0.0,0.0,0.0,0.00198,0.0,0.03404,0.0,...,0.0,0.00114,0.14391,0.0000,0.0,0.0,0.0,0.0,0.0,0.01833


### Looking at species in metagenomic data

In [11]:
mgx_species = mgx[mgx['taxa'].str.contains("\|s__")] # keep families genera
mgx_species = mgx_species[~mgx_species['taxa'].str.contains("\|t__")] # remove genera

mgx_species["taxa"] = mgx_species['taxa'].str.split("\|s__").str[-1]

In [12]:
mgx_species = mgx_species[~mgx_species.taxa.str.contains("_unclassified")]
mgx_species = mgx_species[~mgx_species.taxa.str.contains("_noname")]
mgx_species = mgx_species[~mgx_species.taxa.str.contains("virus")]
mgx_species = mgx_species[~mgx_species.taxa.str.contains("Candidatus")]
mgx_species = mgx_species[~mgx_species.taxa.str.contains("candidate")]

In [13]:
mgx_species = mgx_species.groupby(['taxa']).sum().reset_index() 

In [14]:
mgx_species.head()

,taxa,G69146,G69147,G69148,G69149,G69150,G69152,G69153,G69154,G69155,...,G80612,G80613,G80614,G80615,G80616,G80619,G80620,G80621,G80623,G80624
0,Abiotrophia_defectiva,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acidaminococcus_fermentans,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.88186,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acidaminococcus_intestini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11584,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acidaminococcus_sp_HPA0509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acinetobacter_baumannii,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
len(mgx_species)

519

### Amplicon data

In [17]:
# read in 16S data

amp_genus = pd.read_csv("karalia_dada2_genera.csv")
amp_genus.rename(columns={'genus': 'taxa'}, inplace=True)

In [18]:
# amp_genus = amp[amp['taxa'].str.contains("\|g__")] # keep genera
# amp_genus = amp_genus[~amp_genus['taxa'].str.contains("\|s__")] # keep species
# amp_genus["taxa"] = amp_genus['taxa'].str.split("\|g__").str[-1]
# amp_genus["taxa"] = amp_genus['taxa'].str.split("\|s__").str[0]#

In [20]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
amp_genus = amp_genus[~amp_genus.taxa.str.contains("_unclassified")]
amp_genus = amp_genus[~amp_genus.taxa.str.contains("_noname")]
amp_genus = amp_genus[~amp_genus.taxa.str.contains("virus")]
amp_genus = amp_genus[~amp_genus.taxa.str.contains("Candidatus")]
amp_genus = amp_genus[~amp_genus.taxa.str.contains(r'[0-9]')]
amp_genus = amp_genus[~amp_genus.taxa.str.contains("group")]
amp_genus['taxa'].replace('', np.nan, inplace=True)
amp_genus.dropna(subset=['taxa'], inplace=True)

In [21]:
amp_genus.taxa = amp_genus.taxa = amp_genus.taxa.str.strip('[]')

In [22]:
# remove digits
amp_genus.taxa = amp_genus[~amp_genus.taxa.str.contains(r'\d')]

In [23]:
amp_genus = amp_genus.groupby(['taxa']).sum().reset_index() 

In [24]:
amp_genus.head()

,taxa,G63225,G63227,G63228,G63229,G63230,G63231,G63232,G63234,G63236,...,G76485,G76486,G76487,G76488,G76489,G76490,G76491,G76492,G76493,G76494
0,Abiotrophia,0.0,0.000327,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Absiella,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acetanaerobacterium,0.0,0.000000,0.0,0.000187,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acetobacter,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Achromobacter,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### amplicon family

In [23]:
amp_family = amp[amp['taxa'].str.contains("\|f__")] # keep families genera
amp_family = amp_family[~amp_family['taxa'].str.contains("\|g__")] # remove genera

amp_family["taxa"] = amp_family['taxa'].str.split("\|f__").str[-1]

In [28]:
amp_family = amp_family[~amp_family.taxa.str.contains("_unclassified")]
amp_family = amp_family[~amp_family.taxa.str.contains("_noname")]
amp_family = amp_family[~amp_family.taxa.str.contains("virus")]
amp_family = amp_family[~amp_family.taxa.str.contains("Candidatus")]
amp_family = amp_family[~amp_family.taxa.str.contains(r'[0-9]')]
amp_family = amp_family[~amp_family.taxa.str.contains("group")]
amp_family['taxa'].replace('', np.nan, inplace=True)
amp_family.dropna(subset=['taxa'], inplace=True)

In [29]:
amp_family = amp_family.groupby(['taxa']).sum().reset_index() 

In [30]:
amp_family.sort_values(by = "taxa", ascending=True, inplace=True)

In [31]:
amp_family

,taxa,3101193,3107294,3113022,3107293,3101190,3000144,3113030,3101046,3107810,...,3103862,3119985,3103949,3106651,3117543,3117546,3119987,3119973,3116926,3116924
0,Actinomycetaceae,0.000441,0.000147,0.000058,0.000000,0.000211,0.052666,0.000211,0.030423,0.007408,...,0.000217,0.000014,0.001036,0.000000,0.000040,0.000010,0.000029,0.000000,0.000164,0.000163
1,Aerococcaceae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000081,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000059,0.000000,0.000000,0.000000
2,Aeromonadaceae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Alcaligenaceae,0.000000,0.000000,0.023975,0.000000,0.000053,0.000000,0.052781,0.000000,0.000000,...,0.135141,0.000297,0.000000,0.002109,0.070819,0.051308,0.007244,0.008285,0.000033,0.000000
4,Anaeroplasmataceae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Bacteroidaceae,0.000126,0.000000,0.406499,0.012675,0.000475,0.000000,0.361795,0.000546,0.023871,...,0.000434,0.148948,0.000435,0.123858,0.138286,0.214328,0.178026,0.320707,0.000164,0.000436
6,Bifidobacteriaceae,0.227284,0.000735,0.009821,0.166144,0.037663,0.061027,0.012075,0.108186,0.201255,...,0.030369,0.019675,0.002407,0.001828,0.006308,0.001989,0.001320,0.018036,0.285443,0.269260
7,Campylobacteraceae,0.000315,0.001249,0.000000,0.001499,0.000000,0.002280,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000059,0.000110,0.000000,0.000109
8,Carnobacteriaceae,0.002142,0.000147,0.000116,0.010631,0.001056,0.001629,0.000138,0.001637,0.000617,...,0.000217,0.000189,0.005650,0.000281,0.000040,0.000087,0.000015,0.000151,0.000655,0.000436
9,Catabacteriaceae,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
# ratio of unclassified genera for 16S profiling
sum(amplicon.taxa.str.contains("_unclassified| unidentified| uncultured| group"))/amplicon.shape[0]

In [ ]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy at genus level
#### calculate number of taxa in both vs. either one

In [25]:
# calculate number of taxa in both vs. either one

In [26]:
amplicon_genera_list = set(amp_genus["taxa"])
mgx_genera_list = set(mgx_genus["taxa"])

In [27]:
intersection_genera = amplicon_genera_list.intersection(mgx_genera_list)
union_genera = amplicon_genera_list.union(mgx_genera_list)

In [28]:
len(intersection_genera)
len(amplicon_genera_list)-len(intersection_genera)
len(mgx_genera_list)-len(intersection_genera)

158

197

40

In [29]:
amplicon_genera = amplicon_genera_list.difference(intersection_genera)
mgx_genera = mgx_genera_list.difference(intersection_genera)

In [30]:
len(amplicon_genera)
len(mgx_genera)

197

40

### looking for the intersection of taxonomy at family level

In [44]:
amplicon_family_list = set(amp_family["taxa"])
mgx_family_list = set(mgx_family["taxa"])

intersection_family = amplicon_family_list.intersection(mgx_family_list)
union_family = amplicon_family_list.union(mgx_family_list)

len(intersection_family)
len(amplicon_family_list)-len(intersection_family)
len(mgx_family_list)-len(intersection_family)

39

16

51

In [45]:
amplicon_family = amplicon_family_list.difference(intersection_family)
mgx_family = mgx_family_list.difference(intersection_family)

len(amplicon_family)
len(mgx_family)

16

51

### creating long-form dataframe with data from both sequencing methods

In [33]:
# only keeping samples we have both mgx and 16S profiles for
samples_intersect = set(mgx_genus.columns.values).intersection(set(amp_genus.columns.values))

In [34]:
# remove mothers
children_intersect = [id for id in samples_intersect if not id.startswith("M")]

In [51]:
len(amp_genus.columns.values)

1836

In [50]:
mgx_genus.columns.values

array(['G69146', 'G69147', 'G69148', 'G69149', 'G69150', 'G69151',
       'G69152', 'G69153', 'G69154', 'G69155', 'G69156', 'G69157',
       'G69158', 'G69159', 'G69160', 'G69161', 'G69162', 'G69163',
       'G69164', 'G69165', 'G69166', 'G69167', 'G69168', 'G69169',
       'G69170', 'G69171', 'G69172', 'G69173', 'G69174', 'G69175',
       'G69176', 'G69177', 'G69178', 'G69179', 'G69180', 'G69181',
       'G69182', 'G69183', 'G69184', 'G69185', 'G69186', 'G69187',
       'G69188', 'G69189', 'G69190', 'G69191', 'G69192', 'G69193',
       'G69194', 'G69195', 'G69196', 'G69197', 'G69198', 'G69199',
       'G69201', 'G69202', 'G69203', 'G69204', 'G69205', 'G69206',
       'G69207', 'G69208', 'G69209', 'G69210', 'G69211', 'G69212',
       'G69213', 'G69214', 'G69215', 'G69216', 'G69217', 'G69218',
       'G69219', 'G69221', 'G69222', 'G69223', 'G69224', 'G69225',
       'G69226', 'G69227', 'G69228', 'G69229', 'G69230', 'G69231',
       'G69232', 'G69233', 'G69234', 'G69235', 'G69236', 'G692

In [43]:
samples_intersect

{'taxa'}

In [35]:
mgx = mgx_genus[children_intersect]
amplicon = amp_genus[children_intersect]

In [36]:
# confirm that we now have the same samples
(amp_genus.columns.values) == (mgx_genus.columns.values)

/Users/danielle/miniconda2/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


False

In [ ]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [ ]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [ ]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [ ]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [ ]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [ ]:
merged_taxa.sample(10)

In [ ]:
merged_taxa.fillna(0)

In [ ]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [ ]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [ ]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]

In [ ]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [ ]:
mean_taxa_abund.sample(10)

In [ ]:
mean_taxa_abund.to_csv('taxa_difference.csv')

## making giant dataframe of abundances

In [ ]:
amp_trans = amplicon.set_index("taxa").transpose()

In [ ]:
amp_trans.reset_index(level=0, inplace=True)

In [ ]:
amp_trans.rename(columns = {'index':'sampleid'}, inplace = True) 

In [ ]:
amp_trans["uid"] = amp_trans["sampleid"].astype(str)+'-amp'

In [ ]:
amp_trans["method"] = "amp"

In [ ]:
amp_trans.head()

In [ ]:
mgx_trans = mgx.set_index("taxa").transpose()

In [ ]:
mgx_trans.reset_index(level=0, inplace=True)

In [ ]:
mgx_trans.rename(columns = {'index':'sampleid'}, inplace = True)

In [ ]:
mgx_trans["uid"] = mgx_trans["sampleid"].astype(str)+'-mgx'

In [ ]:
mgx_trans["method"] = "mgx"

In [ ]:
mgx_trans.head()

In [ ]:
concat_df = pd.concat([mgx_trans,amp_trans], sort=True).reset_index(drop = True)

In [ ]:
len(concat_df)

In [ ]:
age = pd.read_csv("~/Documents/thesis/analysis/metadatawide.csv")

In [ ]:
age = age[['sample','childAgeMonths']]

In [ ]:
age["sample"] = age["sample"].str.replace("_",'-')
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [ ]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["childAgeMonths"]
    agedict[row["sampleid"]]= age_months

In [ ]:
concat_df["AgeMonths"]= concat_df["sampleid"].map(agedict)

In [ ]:
cols_to_order = ['uid', 'sampleid',"method", "AgeMonths"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [ ]:
concat_df.sample(15)

In [ ]:
concat_df.to_csv('transposed_mgxamp_df.csv')

In [ ]:
pwd()